In [ ]:
%%shell
git clone https://github.com/plarotta/vision-encoder.git
mv /content/vision-encoder/transfer_learner/ /content/
rm -rf /content/vision-encoder/
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zxFDf6wqacM4EfMzLp2YD7kI9g-3KtJk' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zxFDf6wqacM4EfMzLp2YD7kI9g-3KtJk" -O UR5_images_2.zip && rm -rf /tmp/cookies.txt
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1TAMiAfoJIyFmvGo5gLzxftRHprF60PzG' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1TAMiAfoJIyFmvGo5gLzxftRHprF60PzG" -O UR5_positions_2.csv && rm -rf /tmp/cookies.txt
unzip UR5_images_2.zip
mv UR5_images_2/ images/
pip install wandb -qq

In [10]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
import torch
import os
from skimage import io
from torchvision.transforms import v2
from matplotlib import pyplot as plt
from torchvision.models.resnet import ResNet18_Weights
from torchvision.models import resnet18
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transfer_learner.source.model import RobotNet
from transfer_learner.source.robot_dataset import RobotImageDataset
from transfer_learner.utils.trainers import train_one_epoch, validate_one_epoch
import wandb

In [ ]:
robotdata = RobotImageDataset('/content/UR5_positions_2.csv', '/content/images', multi_input=True)


def train_one_epoch(tr_loader, model, loss_func, optimizer, device):
    running_loss = 0.0

    for i, data in enumerate(tr_loader):
        im1,im2,im3 = data['images']
        y = data['joint_values'].float()
        im1,im2,im3, y = im1.to(device), im2.to(device), im3.to(device), y.to(device)
        optimizer.zero_grad()
        preds = model(im1, im2, im3).float()
        loss = loss_func(preds, y)
        running_loss +=loss.item()
        loss.backward()
        optimizer.step()

    train_loss = running_loss/(i+1)

    return(train_loss)

def validate_one_epoch(val_loader, model, loss_func, optimizer, device):
    running_loss = 0.0

    with torch.no_grad():
        for i, data in enumerate(val_loader):
            im1,im2,im3 = data['images']
            y = data['joint_values'].float()
            im1,im2,im3, y = im1.to(device), im2.to(device), im3.to(device), y.to(device)
            optimizer.zero_grad()
            preds = model(im1, im2, im3).float()
            loss = loss_func(preds, y)
            running_loss +=loss.item()

    val_loss = running_loss/(i+1)

    return(val_loss)

def main():
    run = wandb.init()
    LEARNING_RATE = 1e-4
    BATCH_SIZE = 32
    EPOCH = 5
    IN_DIM1 = 128
    IN_DIM2 = 32


    # Check if GPU is available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Params
    traindata, validdata = random_split(robotdata, [0.8,0.2])
    trainloader = DataLoader(traindata, batch_size=BATCH_SIZE, shuffle=True)
    validloader = DataLoader(validdata, batch_size=BATCH_SIZE, shuffle=True)
    multi_model = RobotNet(IN_DIM1, IN_DIM2,n_inputs=3)


    # Initialize
    multi_model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(multi_model.parameters(), lr=LEARNING_RATE)

    for epoch in range(1,EPOCH):

        tr_loss = train_one_epoch(trainloader, multi_model, criterion, optimizer, device)
        val_loss = validate_one_epoch(validloader, multi_model, criterion, optimizer, device)
        wandb.log(
            {
                "epoch": epoch,
                "train_loss": tr_loss,
                "val_loss": val_loss,
            }
        )
    run.finish()

